In [1]:
from IPython.display import clear_output

In [2]:
!pip install sentence-transformers
!pip install accelerate
!pip install bitsandbytes
!pip install peft
!pip install datasets
!pip install transformers
!pip install loralib
!pip install einops
!pip install googletrans==3.1.0a0
!pip install huggingface_hub

clear_output()

In [3]:
import os
import shutil
import torch
import transformers
from googletrans import Translator
from datasets import load_dataset, Dataset
from torch.utils.data import Dataset as TorchDataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig

In [ ]:
MODEL_NAME = "vilm/vinallama-7b-chat"
PROJECT_NAME = "Group01-ML-Project"
FINE_TUNE_NAME = "math-vinallama-7b-chat"
USERNAME = "Namronaldo2004"
HUGGINGFACE_HUB_REPO = USERNAME + "/" + PROJECT_NAME
FINE_TUNE_REPO = USERNAME + "/" + FINE_TUNE_NAME
EMAIL = "thanhdoan0910@gmail.com"
REPO_ACCESS_TOKEN = os.getenv("REPO_ACCESS_TOKEN")
FINE_TUNE_ACCESS_TOKEN = os.getenv("FINE_TUNE_ACCESS_TOKEN")

In [6]:
from huggingface_hub.hf_api import HfFolder
from huggingface_hub import Repository, HfApi

HfFolder.save_token(REPO_ACCESS_TOKEN)

In [7]:
repo = Repository(local_dir = PROJECT_NAME,
                  git_user = USERNAME,
                  git_email = EMAIL,
                  clone_from = HUGGINGFACE_HUB_REPO)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Namronaldo2004/Group01-ML-Project into local empty directory.


Download file vinallama-7b-chat/back-up-fine-tune-model/checkpoint-100/optimizer.pt:   0%|          | 6.96k/81…

Download file vinallama-7b-chat/back-up-fine-tune-model/runs/Dec27_05-03-14_681e1741ce62/events.out.tfevents.1…

Download file datasets/data-00000-of-00001.arrow:   0%|          | 31.0k/215M [00:00<?, ?B/s]

Clean file vinallama-7b-chat/back-up-fine-tune-model/runs/Dec27_05-03-14_681e1741ce62/events.out.tfevents.1735…

...

Clean file vinallama-7b-chat/pretrained_model/adapter_model.safetensors:   0%|          | 1.00k/153M [00:00<?,…

Clean file math-vinallama-7b-chat/pretrained_model/adapter_model.safetensors:   0%|          | 1.00k/153M [00:…

Clean file datasets/data-00000-of-00001.arrow:   0%|          | 1.00k/215M [00:00<?, ?B/s]

Clean file vinallama-7b-chat/back-up-fine-tune-model/checkpoint-100/optimizer.pt:   0%|          | 1.00k/813M …

Clean file vinallama-7b-chat/fine-tune-model/checkpoint-100/optimizer.pt:   0%|          | 1.00k/813M [00:00<?…

In [10]:
class MicrosoftMathDataset(TorchDataset):
    def __init__ (self, start_sample_index = 0, num_get_samples = None, tokenizer = None):
        raw_data = None
        if ("datasets" in os.listdir(PROJECT_NAME)):
            directory = f"{PROJECT_NAME}/datasets"

            arrow_file = [file for file in os.listdir(directory) if file.endswith(".arrow")][0]
            raw_data = Dataset.from_file(os.path.join(directory, arrow_file))

        else:
            raw_data = load_dataset("microsoft/orca-math-word-problems-200k")['train']
            raw_data.save_to_disk(f"{PROJECT_NAME}/datasets")
            repo.push_to_hub(commit_message = f"Push the microsoft/orca-math-word-problems-200k dataset")

        if (num_get_samples is not None):
            raw_data = raw_data[start_sample_index: (start_sample_index + num_get_samples)]
        else:
            raw_data = raw_data[start_sample_index:]

        self.question = raw_data['question']
        self.answer = raw_data['answer']

        self.translator = Translator()
        self.tokenizer = tokenizer

    def __len__ (self):
        return len(self.question)

    def __getitem__(self, index):
        question = self.question[index].replace('\n\n', ' ').replace('\n', ' ')
        answer = self.answer[index].replace('\n\n', ' ').replace('\n', ' ')

        training_sample = self.generate_and_tokenize_prompt(question, answer)
        return training_sample

    def generate_and_tokenize_prompt(self, question, answer):
        def generate_prompt(question, answer):
            vn_question = self.translator.translate(question, src = 'en', dest = 'vi').text
            vn_answer = self.translator.translate(answer, src = 'en', dest = 'vi').text

            return f"""
            <|im_start|>system
            Bạn là một chuyên gia về toán. Khi nhận được yêu cầu từ người dùng, dựa vào những kiến thức mà bạn đã có, hãy trả lời người dùng một cách nhất quán, đầy đủ nhưng tránh dư thừa thông tin, và hơn hết nội dung câu trả lời phải chính xác nhất
            <|im_end|>
            <|im_start|>user
            ### Câu hỏi:
            {vn_question}
            ### Câu trả lời:
            <|im_end|>
            <|im_start|>assistant
            {vn_answer}
            """.strip()

        full_prompt = generate_prompt(question, answer)

        tokenized_full_prompt = self.tokenizer(
            full_prompt,
            padding = True,
            truncation = True
        )

        return tokenized_full_prompt

In [11]:
class MyMathDataset(TorchDataset):
    def __init__ (self, start_sample_index = 0, num_get_samples = None, tokenizer = None):
        raw_data = None
        if ("new_dataset" in os.listdir(PROJECT_NAME)):
            directory = f"{PROJECT_NAME}/new_dataset"

            arrow_file = [file for file in os.listdir(directory) if file.endswith(".arrow")][0]
            raw_data = Dataset.from_file(os.path.join(directory, arrow_file))

        else:
            raw_data = load_dataset("Mels22/Vietnamese-Intermediate-Reality-Math-Problems")['train']
            raw_data.save_to_disk(f"{PROJECT_NAME}/new_dataset")
            repo.push_to_hub(commit_message = f"Push the Mels22/Vietnamese-Intermediate-Reality-Math-Problems dataset")

        if (num_get_samples is not None):
            raw_data = raw_data[start_sample_index: (start_sample_index + num_get_samples)]
        else:
            raw_data = raw_data[start_sample_index:]

        self.question = raw_data['problem']
        self.answer = raw_data['solution']

        self.tokenizer = tokenizer

    def __len__ (self):
        return len(self.question)

    def __getitem__(self, index):
        
        question = self.question[index]
        answer = self.answer[index]

        training_sample = self.generate_and_tokenize_prompt(question, answer)
        return training_sample

    def generate_and_tokenize_prompt(self, question, answer):
        def generate_prompt(question, answer):
            return f"""
            <|im_start|>system
            Bạn là một chuyên gia về toán. Khi nhận được yêu cầu từ người dùng, dựa vào những kiến thức mà bạn đã có, hãy trả lời người dùng một cách nhất quán, đầy đủ nhưng tránh dư thừa thông tin, và hơn hết nội dung câu trả lời phải chính xác nhất
            <|im_end|>
            <|im_start|>user
            ### Câu hỏi:
            {question}
            ### Câu trả lời:
            <|im_end|>
            <|im_start|>assistant
            {answer}
            """.strip()

        full_prompt = generate_prompt(question, answer)

        tokenized_full_prompt = self.tokenizer(
            full_prompt,
            padding = True,
            truncation = True
        )

        return tokenized_full_prompt

In [12]:
class MathLLM:
    def __init__ (self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = None
        self.model_name = None
        self.tokenizer = None
        self.generation_config = None

    def get_model (self, model_name, generation_config = None):
        # Get folders in the repo
        api = HfApi()
        files = api.list_repo_files(repo_id = HUGGINGFACE_HUB_REPO)
        folders = {file.split('/')[0] for file in files if '/' in file}

        if (model_name.split('/')[-1] in folders):
            bnb_config = BitsAndBytesConfig(
                load_in_4bit = True,
                bnb_4bit_use_double_quant = True,
                bnb_4bit_quant_type = "nf4",
                bnb_4bit_compute_dtype = torch.bfloat16
            )

            if ("fine-tune-model" in os.listdir(f"{PROJECT_NAME}/{model_name.split('/')[-1]}")):
                directory = f"{PROJECT_NAME}/{model_name.split('/')[-1]}/fine-tune-model"
                checkpoint = [f for f in os.listdir(directory) if f.startswith("checkpoint")][0]
                checkpoint_path = os.path.join(directory, checkpoint)
                
                self.model = AutoModelForCausalLM.from_pretrained(
                    checkpoint_path,
                    device_map = "auto",
                    trust_remote_code = True,
                    quantization_config = bnb_config
                )
                self.model = PeftModel.from_pretrained(self.model, checkpoint_path)
                self.model.gradient_checkpointing_enable()
                self.model = prepare_model_for_kbit_training(self.model)
            else:
                self.model = AutoModelForCausalLM.from_pretrained(
                    f"{PROJECT_NAME}/{model_name.split('/')[-1]}/pretrained_model",
                    device_map = "auto",
                    trust_remote_code = True,
                    quantization_config = bnb_config
                )
                self.model = PeftModel.from_pretrained(self.model, f"{PROJECT_NAME}/{model_name.split('/')[-1]}/pretrained_model")
                self.model.gradient_checkpointing_enable()
                self.model = prepare_model_for_kbit_training(self.model)

            # Load tokenizer and generation_config from local directories
            self.tokenizer = AutoTokenizer.from_pretrained(f"{PROJECT_NAME}/{model_name.split('/')[-1]}/tokenizer")
            self.generation_config = GenerationConfig.from_pretrained(f"{PROJECT_NAME}/{model_name.split('/')[-1]}/generation_config")

            # Assign the loaded generation_config to the model
            self.model.generation_config = self.generation_config
            self.model_name = model_name
        else:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit = True,
                bnb_4bit_use_double_quant = True,
                bnb_4bit_quant_type = "nf4",
                bnb_4bit_compute_dtype = torch.bfloat16
            )

            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                device_map = "auto",
                trust_remote_code = True,
                quantization_config = bnb_config
            )
            self.model_name = model_name

            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.tokenizer.pad_token = self.tokenizer.eos_token

            self.model.gradient_checkpointing_enable()
            self.model = prepare_model_for_kbit_training(self.model)

            config = LoraConfig (
                r = 16,
                lora_alpha = 32,
                target_modules = [
                    "q_proj",
                    "up_proj",
                    "o_proj",
                    "k_proj",
                    "down_proj",
                    "gate_proj",
                    "v_proj"
                ],
                lora_dropout = 0.05,
                bias = "none",
                task_type = "CAUSAL_LM"
            )

            self.model = get_peft_model(self.model, config)

            if (generation_config is None):
                self.generation_config = self.model.generation_config
                self.generation_config.temperature = 0.7
                self.generation_config.top_p = 0.7
                self.generation_config.num_return_sequences = 1
                self.generation_config.pad_token_id = self.tokenizer.eos_token_id
                self.generation_config.eos_token_id = self.tokenizer.eos_token_id
            else:
                self.generation_config = generation_config

            self.model.save_pretrained(f"{PROJECT_NAME}/{model_name.split('/')[-1]}/pretrained_model")
            self.tokenizer.save_pretrained(f"{PROJECT_NAME}/{model_name.split('/')[-1]}/tokenizer")
            self.generation_config.save_pretrained(f"{PROJECT_NAME}/{model_name.split('/')[-1]}/generation_config")

            repo.push_to_hub(commit_message = "Upload pretrained model, tokenizer and generation config")

    def fine_tune_LLM (self, train_data, num_epoch = 1, batch_size = 256, learning_rate = 2e-4):
        if (self.model is None or self.tokenizer is None):
            raise Exception ("Please provide pretrained model before fine-tuning!")

        self.model.train()
        for name, param in self.model.named_parameters():
            if param.dtype in [torch.float32, torch.float16, torch.bfloat16, torch.complex64, torch.complex128]:
                param.requires_grad = True

        source_dir = f"{PROJECT_NAME}/{self.model_name.split('/')[-1]}/fine-tune-model"
        dest_dir = f"{PROJECT_NAME}/{self.model_name.split('/')[-1]}/back-up-fine-tune-model"

        if os.path.exists(source_dir):
            # Create a back up directory
            if not os.path.exists(dest_dir):
                os.mkdir(dest_dir)
    
            # Delete all files in the back up directory
            for item in os.listdir(dest_dir):
                item_path = os.path.join(dest_dir, item)
                if os.path.isfile(item_path):
                    os.remove(item_path)
                elif os.path.isdir(item_path):
                    shutil.rmtree(item_path)
    
            # Move previously fine-tuned model to the back up directory
            for item_name in os.listdir(source_dir):
                source_path = os.path.join(source_dir, item_name)
                dest_path = os.path.join(dest_dir, item_name)
                
                shutil.move(source_path, dest_path)
        
        training_args = transformers.TrainingArguments(
            output_dir = source_dir,
            per_device_train_batch_size = 1,
            gradient_accumulation_steps = batch_size,
            num_train_epochs = num_epoch,
            learning_rate = learning_rate,
            fp16 = True,
            save_total_limit = 1,
            logging_steps = 1,
            optim = "paged_adamw_8bit",
            lr_scheduler_type = "cosine",
            warmup_ratio = 0.05,
        )

        trainer = transformers.Trainer(
            model = self.model,
            train_dataset = data,
            args = training_args,
            data_collator = transformers.DataCollatorForLanguageModeling(self.tokenizer, mlm = False)
        )

        self.model.config.use_cache = False
        trainer.train()
        repo.push_to_hub(commit_message = "Completed training!")

    def generate_answer (self, query):
        def get_prompt (system_prompt, query):
            return f"""
            <|im_start|>system
            {system_prompt}
            <|im_end|>
            <|im_start|>user
            ### Yêu cầu:
            {query}
            ### Câu trả lời:
            <|im_end|>
            <|im_start|>assistant
            """.strip()

        prompt = get_prompt(
            system_prompt = "Bạn là một chuyên gia về toán. Khi nhận được yêu cầu từ người dùng, dựa vào những kiến thức mà bạn đã có, hãy trả lời người dùng một cách nhất quán, đầy đủ nhưng tránh dư thừa thông tin, và hơn hết nội dung câu trả lời phải chính xác nhất",
            query = query
        )

        encoding = self.tokenizer(prompt, return_tensors = "pt").to(self.device)
        with torch.inference_mode():
            outputs = self.model.generate(
                input_ids = encoding.input_ids,
                attention_mask = encoding.attention_mask,
                generation_config = self.generation_config
            )

        return self.tokenizer.decode(outputs[0], skip_special_tokens = True)

    def deploy_fine_tune_model (self):
        HfFolder.save_token(FINE_TUNE_ACCESS_TOKEN)
        fine_tune_repo = Repository(local_dir = FINE_TUNE_NAME,
                                    git_user = USERNAME,
                                    git_email = EMAIL,
                                    clone_from = FINE_TUNE_REPO)
    
        # Delete all files in the back up directory
        for item in os.listdir(FINE_TUNE_NAME):
            item_path = os.path.join(FINE_TUNE_NAME, item)
            if os.path.isfile(item_path) and item_path != f'{FINE_TUNE_NAME}/.gitattributes':
                os.remove(item_path)
        
        deploy_file_paths = []
        tokenizer_dir = f"{PROJECT_NAME}/{self.model_name.split('/')[-1]}/tokenizer"
        generation_config_dir = f"{PROJECT_NAME}/{self.model_name.split('/')[-1]}/generation_config"
        fine_tune_dir = f"{PROJECT_NAME}/{self.model_name.split('/')[-1]}/fine-tune-model"
    
        for file in os.listdir(tokenizer_dir):
            deploy_file_paths.append(os.path.join(tokenizer_dir, file))
    
        for file in os.listdir(generation_config_dir):
            deploy_file_paths.append(os.path.join(generation_config_dir, file))
    
        checkpoint = [file for file in os.listdir(fine_tune_dir) if file.startswith("checkpoint")][0]
        checkpoint_path = os.path.join(fine_tune_dir, checkpoint)
        for file in os.listdir(checkpoint_path):
            deploy_file_paths.append(os.path.join(checkpoint_path, file))
    
        for file_path in deploy_file_paths:
            shutil.copy(file_path, FINE_TUNE_NAME)
    
        fine_tune_repo.push_to_hub(commit_message = "Update fine-tuned model")
    
        HfFolder.save_token(REPO_ACCESS_TOKEN)
        shutil.rmtree(FINE_TUNE_NAME)

In [13]:
math_llm = MathLLM()
math_llm.get_model(model_name = MODEL_NAME)

data = MyMathDataset(start_sample_index = 0, tokenizer = math_llm.tokenizer)
math_llm.fine_tune_LLM(train_data = data, num_epoch = 10, batch_size = 100, learning_rate = 2e-4)

# response = math_llm.generate_answer(query = "Hãy cho tôi biết kết quả của phép tính 45 * 6 là bao nhiêu?")
# print(response)

math_llm.deploy_fine_tune_model()

config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

...

Upload file training_args.bin:   0%|          | 1.00/5.12k [00:00<?, ?B/s]

Upload file rng_state.pth:   0%|          | 1.00/13.9k [00:00<?, ?B/s]

Upload file scheduler.pt:   0%|          | 1.00/1.04k [00:00<?, ?B/s]

To https://huggingface.co/Namronaldo2004/math-vinallama-7b-chat
   68b3bbd..cc14b49  main -> main



wandb: 🚀 View run Group01-ML-Project/vinallama-7b-chat/fine-tune-model at: https://wandb.ai/thanhdoan0910-i-h-c-qu-c-gia-tp-hcm/huggingface/runs/nykgg2gv
wandb: Find logs at: wandb/run-20250101_130957-nykgg2gv/logs
